In [1]:
from BRHelper import player_info,extractData
import time
import numpy as np
from bs4 import BeautifulSoup
import requests
from lxml import html
import cPickle,os

In [2]:
players_general_info = player_info() # call function that scrapes general info
validPlayersGeneralInfo = players_general_info[(players_general_info.active_to>1976) & 
                     ((players_general_info.active_to-players_general_info.active_from)>3)]

print validPlayersGeneralInfo.shape

validPlayersGeneralInfo['filename'] = validPlayersGeneralInfo.name.str.replace(' ','_')

(1655, 9)


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
playerDirectory = 'BRData/'
print len(os.listdir(playerDirectory))

BRData = {}
ErrorPlayers = []

for player_idx in range(validPlayersGeneralInfo.shape[0]):
    player_url = validPlayersGeneralInfo.url.values.tolist()[player_idx]
    player_fn = validPlayersGeneralInfo.filename.values.tolist()[player_idx]
    savedPlayers = os.listdir(playerDirectory)
    if player_fn not in savedPlayers:
        page_request = requests.get('http://www.basketball-reference.com' + player_url)
        soup = BeautifulSoup(page_request.text,"lxml")
        try:
            playerData = extractData(soup)
            f = open(playerDirectory+player_fn,'wb')
            cPickle.dump(playerData,f,protocol=cPickle.HIGHEST_PROTOCOL)
            f.close()
            #BRData[player_url] = playerData
        except:
            print player_url,player_fn
            ErrorPlayers.append((player_url,player_idx))
        time.sleep(np.random.randint(10,20))
    
    if (len(savedPlayers)+1)%100==0:
        #save
#         f=open('BRSet.data','wb')
#         cPickle.dump((BRData,ErrorPlayers),f,protocol=cPickle.HIGHEST_PROTOCOL)
#         f.close()
        print (len(savedPlayers)+1)

1620
/players/b/bareajo01.html J.J._Barea
/players/c/capelca01.html Clint_Capela
/players/c/covinro01.html Robert_Covington
/players/c/crabbal01.html Allen_Crabbe
/players/d/dragigo01.html Goran_Dragic
/players/d/dudleja01.html Jared_Dudley
/players/e/exumda01.html Dante_Exum
/players/h/hensojo01.html John_Henson
/players/h/howardw01.html Dwight_Howard
/players/j/jordami01.html Michael_Jordan
/players/l/loveke01.html Kevin_Love
/players/m/mbahalu01.html Luc_Mbah_a_Moute
/players/m/morrima02.html Markieff_Morris
/players/m/mozgoti01.html Timofey_Mozgov
/players/p/plumlmi01.html Miles_Plumlee
/players/r/roberan03.html Andre_Roberson
/players/r/rosede01.html Derrick_Rose
/players/s/sefolth01.html Thabo_Sefolosha
/players/t/thomais02.html Isaiah_Thomas
/players/v/valanjo01.html Jonas_Valanciunas
/players/w/walljo01.html John_Wall


In [ ]:
print len(BRData.keys()),ErrorPlayers